In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from functools import reduce
import datetime
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
def ftrans(num):
    num = int(num)
    if num == 0:
        return 0
    if num == 1000:
        return 1
    if num == 1500:
        return 2
    if num == 2000:
        return 3

def trans(num):
    num = int(num)
    if num == 0:
        return 0
    if num == 1:
        return 1000
    if num == 2:
        return 1500
    if num == 3:
        return 2000

In [4]:
# train_test
train = pd.read_table('data/train/subsidy_train.txt',sep=',',header=-1)
train.columns = ['id','money']
test = pd.read_table('data/test/subsidy_final_test.txt',sep=',',header=-1)
test.columns = ['id']
test['money'] = np.nan
train_test = pd.concat([train,test])

In [6]:
score_train = pd.read_table('data/train/score_train.txt',sep=',',header=-1)
score_train.columns = ['id','college','score']
score_test = pd.read_table('data/test/score_final_test.txt',sep=',',header=-1)
score_test.columns = ['id','college','score']
score_train_test = pd.concat([score_train,score_test])

In [7]:
college_train_test = score_train_test.groupby(["id","college"])["score"].max().unstack(["college"]).fillna(0)
college_name_list = list(college_train_test.columns)
college_train_test = college_train_test.rename(columns=dict(zip(college_name_list,["college_"+str(i) for i in college_name_list]))).reset_index()

In [8]:
college = pd.DataFrame(score_train_test.groupby(['college'])['score'].max()).reset_index()
college.columns = ['college','college_num']
score_train_test = pd.merge(score_train_test, college, how='left',on='college')
score_train_test['college_orderate'] = score_train_test['score']/score_train_test['college_num']
del score_train_test["college"]
#del score_train_test["score"]


In [9]:
score_train_test = pd.merge(score_train_test,college_train_test,how="left",on="id")

In [10]:
train_test = pd.merge(train_test,score_train_test,how='left',on='id')

In [11]:
print train_test

          id   money   score  college_num  college_orderate  college_9  \
0          0     0.0     1.0       2933.0          0.000341        1.0   
1          1     0.0     2.0       2933.0          0.000682        2.0   
2          8     0.0  1565.0       1572.0          0.995547        0.0   
3          9     0.0  1570.0       1572.0          0.998728        0.0   
4         10     0.0     1.0       2301.0          0.000435        0.0   
5         11     0.0     2.0       2301.0          0.000869        0.0   
6         19     0.0     7.0        963.0          0.007269        0.0   
7         20     0.0   356.0        963.0          0.369678        0.0   
8         21     0.0   381.0        963.0          0.395639        0.0   
9         22  1000.0  1734.0       2830.0          0.612721        0.0   
10        23  1500.0  2154.0       2830.0          0.761131        0.0   
11        28  1000.0  2245.0       2830.0          0.793286        0.0   
12        29     0.0  2247.0       283

In [12]:
card_train = pd.read_table('data/train/card_train.txt',sep=',',header=-1)
card_train.columns = ['id','consume','where','how','time','amount','remainder']
card_test = pd.read_table('data/test/card_final_test.txt',sep=',',header=-1)
card_test.columns = ['id','consume','where','how','time','amount','remainder']
card_train_test = pd.concat([card_train,card_test])

In [13]:
print card_train_test
#print card_train_test.groupby("where")["how"].count()

             id consume   where  how                 time  amount  remainder
0          1006   POS消费   地点551   淋浴  2013/09/01 00:00:32    0.50     124.90
1          1006   POS消费   地点551   淋浴  2013/09/01 00:00:32    0.50     124.90
2          1968   POS消费   地点159   淋浴  2013/09/01 00:00:39    0.10     200.14
3          1968   POS消费   地点159   淋浴  2013/09/01 00:00:39    0.10     200.14
4          1406   POS消费   地点660   开水  2013/09/01 00:00:40    0.01     374.42
5          1406   POS消费   地点660   开水  2013/09/01 00:00:40    0.01     374.42
6          1406   POS消费    地点78   其他  2013/09/01 00:00:40    0.60     373.82
7          1406   POS消费    地点78   其他  2013/09/01 00:00:40    0.60     373.82
8         13554   POS消费     地点6   淋浴  2013/09/01 00:00:57    0.50     522.37
9         13554   POS消费     地点6   淋浴  2013/09/01 00:00:57    0.50     522.37
10         5582   POS消费    地点30   开水  2013/09/01 00:01:01    0.27     512.04
11         5582   POS消费    地点30   开水  2013/09/01 00:01:01    0.27     512.04

In [14]:
card = pd.DataFrame(card_train_test.groupby(['id'])["time"].count()).reset_index().rename(columns={"time":"cardActionNum"})

In [15]:
"""剩余金额上，“和”这个概念是不应该的，因此次数越多和就越大，所以一个人的和越大，不一定是体现出消费能力，可能只是比别人多几次消费"""
"""用剩余金额的均值、众数、最小值和最大值还是有一定的理论意义的"""
"""并且剩余金额这个特征是不需要依附其他特征而存在的，本身就可以比较"""
#card['remaindersum'] = card_train_test.groupby(['id'])['remainder'].sum()
card['remainderavg'] = card_train_test.groupby(['id'])['remainder'].mean()
card['remaindermax'] = card_train_test.groupby(['id'])['remainder'].max()
card['remaindermin'] = card_train_test.groupby(['id'])['remainder'].min()
card['remaindermedian'] = card_train_test.groupby(['id'])['remainder'].median()

In [16]:
card_train_test['time'] = card_train_test['time'].apply(lambda x: datetime.datetime.strptime(
        x, "%Y/%m/%d %H:%M:%S").date())

In [17]:
card_how_count = card_train_test.groupby(["id","how"])["amount"].count().unstack(["how"])
card_how_count = card_how_count.rename(columns=dict(zip(card_how_count,[str(i)+"_count" for i in card_how_count])))

card_how_sum = card_train_test.groupby(["id","how"])["amount"].sum().unstack(["how"])
card_how_sum = card_how_sum.rename(columns=dict(zip(card_how_sum,[str(i)+"_sum" for i in card_how_sum])))

card_how_mean = card_train_test.groupby(["id","how"])["amount"].mean().unstack(["how"])
card_how_mean = card_how_mean.rename(columns=dict(zip(card_how_mean,[str(i)+"_mean" for i in card_how_mean])))

card_how_max = card_train_test.groupby(["id","how"])["amount"].max().unstack(["how"])
card_how_max = card_how_max.rename(columns=dict(zip(card_how_max,[str(i)+"_max" for i in card_how_max])))

card_how_min = card_train_test.groupby(["id","how"])["amount"].min().unstack(["how"])
card_how_min = card_how_min.rename(columns=dict(zip(card_how_min,[str(i)+"_min" for i in card_how_min])))

card_how_median = card_train_test.groupby(["id","how"])["amount"].median().unstack(["how"])
card_how_median = card_how_median.rename(columns=dict(zip(card_how_median,[str(i)+"_median" for i in card_how_median])))

In [18]:
card_consume_count = card_train_test.groupby(["id","consume"])["amount"].count().unstack(["consume"])
card_consume_count = card_consume_count.rename(columns=dict(zip(card_consume_count,[str(i)+"_count" for i in card_consume_count])))

card_consume_sum = card_train_test.groupby(["id","consume"])["amount"].sum().unstack(["consume"])
card_consume_sum = card_consume_sum.rename(columns=dict(zip(card_consume_sum,[str(i)+"_sum" for i in card_consume_sum])))

card_consume_mean = card_train_test.groupby(["id","consume"])["amount"].mean().unstack(["consume"])
card_consume_mean = card_consume_mean.rename(columns=dict(zip(card_consume_mean,[str(i)+"_mean" for i in card_consume_mean])))

card_consume_max = card_train_test.groupby(["id","consume"])["amount"].max().unstack(["consume"])
card_consume_max = card_consume_max.rename(columns=dict(zip(card_consume_max,[str(i)+"_max" for i in card_consume_max])))

card_consume_min = card_train_test.groupby(["id","consume"])["amount"].min().unstack(["consume"])
card_consume_min = card_consume_min.rename(columns=dict(zip(card_consume_min,[str(i)+"_min" for i in card_consume_min])))

card_consume_median = card_train_test.groupby(["id","consume"])["amount"].median().unstack(["consume"])
card_consume_median = card_consume_median.rename(columns=dict(zip(card_consume_median,[str(i)+"_median" for i in card_consume_median])))

In [20]:
card_where_count = card_train_test.groupby(["id","where"])["amount"].count().unstack(["where"])
card_where_count = card_where_count.rename(columns=dict(zip(card_where_count,[str(i)+"_count" for i in card_where_count])))

card_where_sum = card_train_test.groupby(["id","where"])["amount"].sum().unstack(["where"])
card_where_sum = card_where_sum.rename(columns=dict(zip(card_where_sum,[str(i)+"_sum" for i in card_where_sum])))

card_where_mean = card_train_test.groupby(["id","where"])["amount"].mean().unstack(["where"])
card_where_mean = card_where_mean.rename(columns=dict(zip(card_where_mean,[str(i)+"_mean" for i in card_where_mean])))

card_where_max = card_train_test.groupby(["id","where"])["amount"].max().unstack(["where"])
card_where_max = card_where_max.rename(columns=dict(zip(card_where_max,[str(i)+"_max" for i in card_where_max])))

card_where_min = card_train_test.groupby(["id","where"])["amount"].min().unstack(["where"])
card_where_min = card_where_min.rename(columns=dict(zip(card_where_min,[str(i)+"_min" for i in card_where_min])))

card_where_median = card_train_test.groupby(["id","where"])["amount"].median().unstack(["where"])
card_where_median = card_where_median.rename(columns=dict(zip(card_where_median,[str(i)+"_median" for i in card_where_median])))

In [21]:
card_time_count = card_train_test.groupby(["id","time"])["amount"].count().unstack(["time"])
card_time_count = card_time_count.rename(columns=dict(zip(card_time_count,[str(i)+"_count" for i in card_time_count])))

card_time_sum = card_train_test.groupby(["id","time"])["amount"].sum().unstack(["time"])
card_time_sum = card_time_sum.rename(columns=dict(zip(card_time_sum,[str(i)+"_sum" for i in card_time_sum])))

card_time_mean = card_train_test.groupby(["id","time"])["amount"].mean().unstack(["time"])
card_time_mean = card_time_mean.rename(columns=dict(zip(card_time_mean,[str(i)+"_mean" for i in card_time_mean])))

card_time_max = card_train_test.groupby(["id","time"])["amount"].max().unstack(["time"])
card_time_max = card_time_max.rename(columns=dict(zip(card_time_max,[str(i)+"_max" for i in card_time_max])))

card_time_min = card_train_test.groupby(["id","time"])["amount"].min().unstack(["time"])
card_time_min = card_time_min.rename(columns=dict(zip(card_time_min,[str(i)+"_min" for i in card_time_min])))

card_time_median = card_train_test.groupby(["id","time"])["amount"].median().unstack(["time"])
card_time_median = card_time_median.rename(columns=dict(zip(card_time_median,[str(i)+"_median" for i in card_time_median])))

In [22]:
card_consume_amount = pd.concat([card_consume_count,card_consume_sum,card_consume_max,card_consume_min,card_consume_median,card_consume_mean],axis=1).reset_index().fillna(0)

In [23]:
card_where_amount = pd.concat([card_where_count,card_where_sum,card_where_max,card_where_min,card_where_median,card_where_mean],axis=1).reset_index().fillna(0)

In [24]:
card_time_amount = pd.concat([card_time_count,card_time_sum,card_time_max,card_time_min,card_time_median,card_time_mean],axis=1).reset_index().fillna(0)

In [25]:
card_how_amount = pd.concat([card_how_count,card_how_sum,card_how_max,card_how_min,card_how_median,card_how_mean],axis=1).reset_index().fillna(0)

## borrow

In [26]:
borrow_train = pd.read_table('data/train/borrow_train.txt',sep=',"',header=None)
borrow_train.columns = ['id','borrowTime','bookName','bookNo']
borrow_test = pd.read_table('data/test/borrow_final_test.txt',sep=',"',header=-1)
borrow_test.columns = ['id','borrowTime','bookName','bookNo']
borrow_train_test = pd.concat([borrow_train,borrow_test])

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


In [27]:
borrow = pd.DataFrame(borrow_train_test.groupby(['id'])["borrowTime"].count()).reset_index().rename(columns={"borrowTime":"borrowActionNum"})

In [28]:
def borrow_find(str):
    flag = 0
    if ((str.find("数学建模")!=-1)|(str.find("建模")!=-1)):
        str = "数学建模"
        flag = 1
    if (str.find("研究生")!=-1)|(str.find("考研")!=-1):
        str = "考研"
        flag = 1
    if ((str.find("数学")!=-1)|(str.find("微积分")!=-1)|(str.find("线性代数")!=-1)|(str.find("数值分析")!=-1)
      |(str.find("概率论")!=-1)|(str.find("数理")!=-1)|(str.find("复变函数")!=-1)|(str.find("应用统计")!=-1)|(str.find("几何")!=-1)|(str.find("方程")!=-1)):
        str = "数学"
        flag = 1
    if str.find("单人研修室")!=-1:
        str = "单人研修室"
        flag = 1
    if str.find("三人研修室")!=-1:
        str = "三人研修室"
        flag = 1
    if str.find("六人研修室")!=-1:
        str = "六人研修室"
        flag = 1
    if (str.find("托福")!=-1)|(str.find("雅思")!=-1)|(str.find("GRE")!=-1)|(str.find("TOEFL")!=-1)|(str.find("耶鲁")!=-1)|(str.find("TOEFL")!=-1):
        str = "出国"
        flag = 1
    if (str.find("英语")!=-1)|(str.find("四六级")!=-1):
        str = "英语"
        flag = 1
    if ((str.find("Java")!=-1)|(str.find("计算机")!=-1)|(str.find("电子")!=-1)|(str.find("数据结构")!=-1)|(str.find("算法")!=-1)
        |(str.find("C语言")!=-1)|(str.find("C++")!=-1)|(str.find("程序设计")!=-1)|(str.find("电路")!=-1)
         |(str.find("系统设计")!=-1)|(str.find("图像处理")!=-1)|(str.find("程序设计")!=-1)|(str.find("数据库")!=-1)
        |(str.find("编程")!=-1)|(str.find("JavaEE")!=-1)|(str.find("HTML")!=-1)|(str.find("四色问题")!=-1)
       |(str.find("Photoshop")!=-1)|(str.find("软件")!=-1)|(str.find("MATLAB")!=-1)|(str.find("SPSS")!=-1)
         |(str.find("单片机")!=-1)|(str.find("代码")!=-1)|(str.find("编译")!=-1)|(str.find("传感器")!=-1)
       |(str.find("Linux")!=-1)|(str.find("软件")!=-1)|(str.find("信号")!=-1)|(str.find("网络")!=-1)
       |(str.find("自动控制")!=-1)|(str.find("JavaScript")!=-1)|(str.find("Android")!=-1)|(str.find("云计算")!=-1)
       |(str.find("CSS")!=-1)|(str.find("MFC")!=-1)|(str.find("嵌入式")!=-1)|(str.find("云计算")!=-1)):
        str = "计算机"
        flag = 1
    if str.find("物理")!=-1:
        str = "物理"
        flag = 1
    if ((str.find("会计")!=-1)|(str.find("经济学")!=-1)):
        str = "经济学"
        flag = 1
    if (str.find("哲学")!=-1)|(str.find("菜根谭")!=-1)|(str.find("心理学")!=-1)|(str.find("精神")!=-1)|(str.find("形而上学")!=-1)|(str.find("思维")!=-1):
        str = "哲学"
        flag = 1
    if str.find("法语")!=-1:
        str = "法语"
        flag = 1
    if str.find("日语")!=-1:
        str = "日语"
        flag = 1
    if str.find("公务员")!=-1:
        str = "公务员考试"
        flag = 1
    if flag ==1:
        return str
    else:
        return "小说"

In [29]:
def borrow_find2(str):
    flag = 0
    if (str.find("小说")!=-1):
        str = "课外阅读"
        flag = 1
    if flag==1:
        return str
    else:
        return "课内学习"

In [30]:
borrow_train_test["bookClassify"] = borrow_train_test["bookName"].apply(borrow_find)
borrow_train_test["bookClassify2"] = borrow_train_test["bookClassify"].apply(borrow_find2)
borrow_train_test["bookTimes"] = borrow_train_test["borrowTime"].apply(lambda x:x.strip('"'))
print borrow_train_test

           id   borrowTime                                           bookName  \
0        9708   2014/2/25"    我的英语日记 wo de ying yu ri ji / (韩)南银英著  (韩)卢炫廷插图"   
1        6956  2013/10/27"         解读联想思维 jie du lian xiang si wei : 联想教父柳传志"   
2       23939    2015/3/8"                         电路分析 dian lu fen xi  刘健主编"   
3       22047  2014/12/29"  现代物流学 xian dai wu liu xue = Contemporary logis...   
4        9076   2014/3/28"  公司法 gong si fa = = Corporation law / 范健, 王建文著 ...   
5        6406   2014/4/13"                             发烧 fa shao  北村著 张润世绘图"   
6       29333   2015/7/14"  数字绘画与创意 shu zi hui hua yu chuang yi (美)Cher Th...   
7       16266   2014/6/26"  周汝昌评说四大名著 zhou ru chang ping shuo si da ming z...   
8       15150   2014/1/11"  管理人力资源 guan li ren li zi yuan 合作伙伴的责任、定位与分工 Ma...   
9       22653   2015/8/30"  面朝大海春暖花开 mian chao da hai chun nuan hua kai 海子...   
10       6895   2014/11/5"  通信电子电路学习指导 tong xin dian zi dian lu xue xi zhi...   
11      21322   2013/9/30"  

In [31]:
borrow_bookClassify_count = borrow_train_test.groupby(["id","bookClassify"])["bookTimes"].count().unstack(["bookClassify"])
borrow_bookClassify_count = borrow_bookClassify_count.rename(columns=dict(zip(borrow_bookClassify_count,
                                                                              [str(i)+"_count" for i in borrow_bookClassify_count]))).reset_index().fillna(0)

borrow_bookClassify2_count = borrow_train_test.groupby(["id","bookClassify2"])["bookTimes"].count().unstack(["bookClassify2"])
borrow_bookClassify2_count = borrow_bookClassify2_count.rename(columns=dict(zip(borrow_bookClassify2_count,[str(i)+"_count" for i 
                                                                                                         in borrow_bookClassify2_count]))).reset_index().fillna(0)

In [32]:
borrow_bookTimes_count = borrow_train_test.groupby(["id","bookTimes"])["bookClassify"].count().unstack(["bookTimes"])
borrow_bookTimes_count = borrow_bookTimes_count.rename(columns=dict(zip(borrow_bookTimes_count,[str(i)+"_count" for i 
                                                                                                         in borrow_bookTimes_count]))).reset_index().fillna(0)

In [34]:
library_train = pd.read_table('data/train/library_train.txt',sep=',',header=None)
library_train.columns = ['id','libraryNo','libraryTime']
library_test = pd.read_table('data/test/library_final_test.txt',sep=',',header=-1)
library_test.columns = ['id','libraryNo','libraryTime']
library_train_test = pd.concat([library_train,library_test])
print library_train_test

            id libraryNo          libraryTime
0         3684         5  2013/09/01 08:42:50
1        14314         6  2013/09/01 08:42:59
2         5108         9  2013/09/01 08:43:32
3         5826         7  2013/09/01 08:44:06
4         7730         5  2013/09/01 08:44:25
5         6668         7  2013/09/01 08:47:24
6        11636         8  2013/09/01 08:48:24
7         4036         6  2013/09/01 08:49:53
8         4050         5  2013/09/01 08:50:01
9         7434         5  2013/09/01 08:50:08
10        7968         7  2013/09/01 08:52:00
11        6956         6  2013/09/01 08:54:13
12       20132         7  2013/09/01 08:57:33
13        3166         5  2013/09/01 09:00:40
14         940         5  2013/09/01 09:01:27
15        7548         7  2013/09/01 09:01:30
16        7262         7  2013/09/01 09:02:21
17        1832         5  2013/09/01 09:10:47
18        9660         7  2013/09/01 09:10:48
19         476         6  2013/09/01 09:12:04
20        7746         6  2013/09/

In [35]:
library = pd.DataFrame(library_train_test.groupby(['id'])["libraryTime"].count()).reset_index().rename(columns={"libraryTime":"libraryActionNum"})

In [36]:
library_train_test['libraryTime'] = library_train_test['libraryTime'].apply(lambda x: datetime.datetime.strptime(
        x, "%Y/%m/%d %H:%M:%S").date())

In [37]:
library_libraryTime_count = library_train_test.groupby(["id","libraryTime"])["libraryNo"].count().unstack(["libraryTime"])
library_libraryTime_count = library_libraryTime_count.rename(columns=dict(zip(library_libraryTime_count,[str(i)+"_library_count" for i 
                                                                                                         in library_libraryTime_count]))).reset_index().fillna(0)

library_libraryNo_count = library_train_test.groupby(["id","libraryNo"])["libraryTime"].count().unstack(["libraryNo"])
library_libraryNo_count = library_libraryNo_count.rename(columns=dict(zip(library_libraryNo_count,[str(i)+"_library_count" for i 
                                                                                                         in library_libraryNo_count]))).reset_index().fillna(0)

In [38]:
train_test = pd.merge(train_test,card,how='left',on='id')
train_test = pd.merge(train_test,card_how_amount,how='left',on='id')
train_test = pd.merge(train_test,card_consume_amount.loc[:,["id","换卡_count","POS消费_mean","卡充值_mean","POS消费_sum"
                                                           ,"POS消费_count","圈存转账_sum","POS消费_max","POS消费_min"
                                                           ,"圈存转账_mean","卡充值_sum" ,"圈存转账_count","卡充值_count"
                                                           ,"卡补办_sum"]],how='left',on='id')
train_test = pd.merge(train_test,card_where_amount.loc[:,["id","地点21_mean","地点213_count","地点72_sum","地点283_sum"
                                                          ,"地点232_mean"]],how='left',on='id')
train_test = pd.merge(train_test,card_time_amount.loc[:,["id","2015-05-21_mean","2014-09-05_mean","2014-09-01_mean","2014-10-27_mean"
                                                          ,"2015-07-19_median","2013-09-02_sum","2014-12-10_min","2014-09-26_mean"
                                                          ,"2015-03-11_sum","2015-02-05_min","2015-07-23_count"]],how='left',on='id')

In [39]:
train_test = pd.merge(train_test,borrow,how='left',on='id')
train_test = pd.merge(train_test,borrow_bookClassify_count,how='left',on='id')
train_test = pd.merge(train_test,borrow_bookClassify2_count,how='left',on='id')

train_test = pd.merge(train_test,borrow_bookTimes_count.loc[:,["id","2015/4/15_count","2014/9/1_count",
                                                         "2013/9/4_count","2014/9/9_count","2013/9/7_count"
                                                               ,"2014/9/2_count","2014/9/10_count",
                                                         "2014/10/27_count","2014/9/3_count","2014/9/5_count"
                                                               ,"2013/12/4_count","2015/6/23_count",
                                                         "2014/12/25_count","2013/9/3_count"]],how='left',on='id')

In [40]:
train_test = pd.merge(train_test,library,how='left',on='id')
train_test = pd.merge(train_test,library_libraryTime_count.loc[:,["id","2014-09-09_library_count","2013-10-07_library_count",
                                                         "2013-09-04_library_count","2015-08-04_library_count","2014-09-03_library_count"
                                                               ,"2013-09-03_library_count"]],how='left',on='id')
train_test = pd.merge(train_test,library_libraryNo_count.loc[:,["id","小门_library_count","出门4_library_count",
                                                         "出门3_library_count"]],how='left',on='id')

## 把这种删除特征的方法下次用到random0.03045上

In [41]:
dropList = ["公务员考试_count","college_18","其他_median","校医院_median","日语_count","college_14",
           "物理_count","college_2","college_1","college_10","校医院_count","其他_min","其他_count",
           "法语_count","三人研修室_count"]
train_test.drop(dropList,axis=1, inplace=True)

In [42]:
dropList2 = ["2013/12/4_count","2015/6/23_count","2013/9/7_count","单人研修室_count","校医院_max","2014/9/2_count",
           "其他_max"]
train_test.drop(dropList2,axis=1, inplace=True)

In [43]:
dropList3 = ["课外阅读_count","校医院_min","英语_count","校医院_mean","考研_count","2014/9/5_count",
           "2014/9/3_count","college_15","2014/9/9_count","2014/12/25_count","2013/9/3_count"]
train_test.drop(dropList3,axis=1, inplace=True)

In [44]:
train = train_test.loc[train_test.money.notnull(),:]
test = train_test.loc[train_test.money.isnull(),:]

In [45]:
print train.shape,test.shape
print train.columns,test.columns

(10885, 133) (10961, 133)
Index([u'id', u'money', u'score', u'college_num', u'college_orderate',
       u'college_9', u'college_19', u'college_6', u'college_3', u'college_4',
       ...
       u'libraryActionNum', u'2014-09-09_library_count',
       u'2013-10-07_library_count', u'2013-09-04_library_count',
       u'2015-08-04_library_count', u'2014-09-03_library_count',
       u'2013-09-03_library_count', u'小门_library_count', u'出门4_library_count',
       u'出门3_library_count'],
      dtype='object', length=133) Index([u'id', u'money', u'score', u'college_num', u'college_orderate',
       u'college_9', u'college_19', u'college_6', u'college_3', u'college_4',
       ...
       u'libraryActionNum', u'2014-09-09_library_count',
       u'2013-10-07_library_count', u'2013-09-04_library_count',
       u'2015-08-04_library_count', u'2014-09-03_library_count',
       u'2013-09-03_library_count', u'小门_library_count', u'出门4_library_count',
       u'出门3_library_count'],
      dtype='object', length

In [46]:
#### Oversample
del test["money"]
Oversampling1000 = train.loc[train.money == 1000]
Oversampling1500 = train.loc[train.money == 1500]
Oversampling2000 = train.loc[train.money == 2000]
for i in range(8):
    train = train.append(Oversampling1000)
for j in range(11):
    train = train.append(Oversampling1500)
for k in range(12):
    train = train.append(Oversampling2000)

# model
train['isnull'] =  train.isnull().sum(axis=1)
test['isnull'] = test.isnull().sum(axis=1)
train = train.set_index(["id"]).fillna(-1)
test = test.set_index(["id"]).fillna(-1)
train['money'] = train['money'].apply(lambda x: ftrans(x))
print train
print test

       money   score  college_num  college_orderate  college_9  college_19  \
id                                                                           
0          0     1.0       2933.0          0.000341        1.0         0.0   
1          0     2.0       2933.0          0.000682        2.0         0.0   
8          0  1565.0       1572.0          0.995547        0.0         0.0   
9          0  1570.0       1572.0          0.998728        0.0         0.0   
10         0     1.0       2301.0          0.000435        0.0         0.0   
11         0     2.0       2301.0          0.000869        0.0         0.0   
19         0     7.0        963.0          0.007269        0.0         0.0   
20         0   356.0        963.0          0.369678        0.0         0.0   
21         0   381.0        963.0          0.395639        0.0         0.0   
22         1  1734.0       2830.0          0.612721        0.0         0.0   
23         2  2154.0       2830.0          0.761131        0.0  

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [47]:
print train[train.money==0].shape
print train[train.money==1].shape
print train[train.money==2].shape
print train[train.money==3].shape

(9325, 133)
(6669, 133)
(5580, 133)
(4602, 133)


In [52]:
def pipeline(seed,max_depth,colsample_bytree,num_round):
    np.random.seed(seed)
    X_train = train.iloc[np.random.permutation(len(train))]
    y_train = X_train.money
    X_train = X_train.drop(['money'],axis=1)
    
    xgb_train   = xgb.DMatrix(X_train,y_train)
    xgb_test = xgb.DMatrix(test)
    
    xgb_params = {
        'booster': 'gbtree',
        #'objective': 'multi:softmax',
        'objective': 'multi:softprob',
        'num_class': 4,
        'max_depth': max_depth,
        'learning_rate': 0.04,
        'colsample_bytree': colsample_bytree,
        'subsample': 0.8,
        'eval_metric': 'merror',
        'silent': 1,
    }
    
    watchlist = [(xgb_train,'train')]
    num_round= num_round
    bst = xgb.train(xgb_params,xgb_train,num_round,watchlist,verbose_eval=False)
    yprob = bst.predict(xgb_test).reshape( test.shape[0], 4 )
    ylabel = np.argmax(yprob, axis=1)
    pd.DataFrame(yprob).to_csv("xgb_prob_seed100_1_0.02783.csv")
    
    pred = pd.DataFrame({'studentid': test.reset_index()['id'],
                             'subsidy': ylabel})
    pred['subsidy'] = pred['subsidy'].apply(lambda x: trans(x))
    pred.to_csv("./xgb_pred_seed100_1_0.02783.csv", index=False)
    #print "数据量:", train.shape[0], ":", pred.shape[0]
    #print "有奖数量:", sum(train['money'] != 0), ":", sum(pred['subsidy'] != 0)
    print "1000奖学金数量：", sum(train['money'] == 1000), sum(pred['subsidy'] == 1000)
    print "1500奖学金数量：", sum(train['money'] == 1500), sum(pred['subsidy'] == 1500)
    print "2000奖学金数量：", sum(train['money'] == 2000), sum(pred['subsidy'] == 2000)
    pd.Series(bst.get_fscore()).reset_index().sort_values(0).to_csv("feature_xgb_prob_seed100_1_0.02783.csv",encoding="GB18030")
    return yprob,sum(pred['subsidy'] == 1000),sum(pred['subsidy'] == 1500),sum(pred['subsidy'] == 2000)

## 执行

In [53]:
prob,a,b,c = pipeline(100,5,0.8,100)

1000奖学金数量： 0 2516
1500奖学金数量： 0 775
2000奖学金数量： 0 279


In [50]:
pd.DataFrame(prob).to_csv("xgb_prob_seed100_1_0.02783.csv")

In [92]:
print a

[[ 0.26054969  0.34977007  0.1663022   0.22337808]
 [ 0.51253694  0.17336488  0.21597905  0.09811909]
 [ 0.65440506  0.18068506  0.09521852  0.06969137]
 ..., 
 [ 0.40733448  0.24945877  0.26073369  0.08247302]
 [ 0.96712822  0.0118625   0.0086104   0.01239888]
 [ 0.88091147  0.05166671  0.02918606  0.03823573]]


## 分析

In [51]:
pd.Series(bst.get_fscore()).reset_index().sort_values(0).to_csv("feature_xgb_prob_seed100_1_0.02783.csv",encoding="GB18030")

NameError: name 'bst' is not defined

In [ ]:
-----------------------------------------
1000,5,0.8,100
数据量: 26176 : 10783
有奖数量: 16851 : 3402
1000奖学金数量： 0 2408
1500奖学金数量： 0 721
2000奖学金数量： 0 273
0.03022
----------------------------------------
1500,5,0.8,100
数据量: 26176 : 10783
有奖数量: 16851 : 3390
1000奖学金数量： 0 2409
1500奖学金数量： 0 718
2000奖学金数量： 0 263
0.02***
----------------------------------------
999,5,0.8,100
数据量: 26176 : 10783
有奖数量: 16851 : 3357
1000奖学金数量： 0 2364
1500奖学金数量： 0 724
2000奖学金数量： 0 269
0.2***
----------------------------------------
4000,5,0.8,100
数据量: 26176 : 10783
有奖数量: 16851 : 3369
1000奖学金数量： 0 2416
1500奖学金数量： 0 712
2000奖学金数量： 0 241
0.03031
-------------------------------------

model4 
---------------------------------------------------------------------
---------------------------------------------------------------------
2432 680 263
-----------------------------
seed = 100,5,0.8,100
1000奖学金数量： 0 2436
1500奖学金数量： 0 678
2000奖学金数量： 0 265
(2436, 678, 265)
0.03041
---------------------------
seed =3480
1000奖学金数量： 0 2434
1500奖学金数量： 0 681
2000奖学金数量： 0 272
(2434, 681, 272)
不到0.03
在另外两个差别不大的情况下，包括两种情况，一是数和分布都一样，那么2000人数增加导致分数下降；另一种是分布不一致，这个要另外考虑了。
--------------------------
1560
数据量: 26176 : 10783
有奖数量: 16851 : 3372
1000奖学金数量： 0 2412
1500奖学金数量： 0 674
2000奖学金数量： 0 286
不到0.03
-----------------------------
2440